In [1]:
import pandas as pd
import numpy as np

In [2]:
num = 54
df_final = pd.DataFrame(np.zeros((12, 58)))
# Read all 54 data files into python dataframe
for i in range(1,num+1):
    file = r"Fruit_Data/"+str(i)+"-ColorConceptAssocFr.xls"
    df = pd.read_excel(file, index_col=None, header = None, parse_cols = [x for x in range(3,16)])
    df = df.transpose()
    df = df.drop([0],axis = 1)
    df.columns = [x for x in range(0,58)]
    df = (df+200) / 400
    df_final = df_final.add(df)
# Average of the ratings per color per subject -- 12x58 matrix
df_final = df_final/num
writer = pd.ExcelWriter('data_clean.xlsx')
df_final.to_excel(writer,'Sheet1')
writer.save()

c:\users\laurent\appdata\local\juliapro-0.6.4.1\python\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: the 'parse_cols' keyword is deprecated, use 'usecols' instead
  


In [3]:
from gurobipy import *
import math
import random
import time

In [4]:
## Define the ratings data and the desired ratings
r = {}
Fruits = [x for x in range(12)] # Later substitute with actual values -- maybe
Colors = [x for x in range(58)]
FC_Map = [(i,j) for i in Fruits for j in Colors]
FC_Map;

In [5]:
df_final.head()

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,0.111435,0.0762037,0.107685,0.108287,0.102315,0.115278,0.108889,0.111435,0.16,0.148611,...,0.38037,0.535231,0.751806,0.618102,0.670231,0.315833,0.262083,0.197176,0.599815,0.854213
1,0.0923148,0.067963,0.129815,0.11287,0.101528,0.185231,0.212083,0.252685,0.38338,0.400972,...,0.137824,0.116528,0.0924074,0.14338,0.128935,0.680972,0.268935,0.511296,0.112269,0.115139
2,0.121667,0.106204,0.128981,0.127037,0.0847685,0.0988426,0.136435,0.0934259,0.0986111,0.112083,...,0.297963,0.431667,0.297083,0.379398,0.220463,0.113889,0.375278,0.446574,0.324815,0.31588
3,0.092037,0.0944907,0.101944,0.0705556,0.094537,0.133472,0.151667,0.102593,0.129074,0.0943056,...,0.389722,0.540648,0.692824,0.780324,0.486574,0.215694,0.24412,0.266759,0.449444,0.725741
4,0.0917593,0.0835648,0.127222,0.115556,0.186111,0.230463,0.253333,0.230046,0.36537,0.297269,...,0.248519,0.365694,0.489861,0.578611,0.414676,0.325278,0.204398,0.125278,0.323194,0.454167


In [6]:
# for i in Fruits:
#     r[i] = [df_final.loc[i][j] for j in Colors ]
for (i,j) in FC_Map:
    r[i,j] = df_final.loc[i,j]
    #print("Fruit",i,"Color",j," --> ",r[i,j])
k = 2             # Desired number of fruits k = 1,2,...p
l = 1             # Desired number of colors l = 1,2,...q
d = {}            # Specify the desired ratings for p fruits and q colors
for i in range(k):
    # d_(k,l) = Desired rate of association for the fruit 'k' with the color 'l'
    d[i] = [random.uniform(0,1) for j in range(l)]
d

{0: [0.5866747092080323], 1: [0.8280081874787142]}

In [7]:
# Model 1
start = time.time()
m = Model("Formulation_2") # Minimizing the max error
##################################################
# variables
##################################################
z = m.addVars(Fruits, Colors,range(k),range(l),vtype = GRB.BINARY, name='z')
t = m.addVar(vtype = GRB.CONTINUOUS, obj = 1,name = 't')


m.update()

# Set Objective
#m.setObjective(sum((d[i][j] - sum(r[p,q]*z[p,q,i,j] for p in Fruits for q in Colors ))*(d[i][j] - sum(r[p,q]*z[p,q,i,j] for p in Fruits for q in Colors )) for i in range(k) for j in range(l)))

m.modelSense = GRB.MINIMIZE
m.update() 

##################################################
# Constraints
##################################################
fruit_con = m.addConstrs(z.sum('*',j,p,q) == 1 for j in Colors for p in range(k) for q in range(l))
color_con = m.addConstrs(z.sum(i,'*',p,q) == 1 for i in Fruits for p in range(k) for q in range(l))

fruit_row_con = m.addConstrs(z.sum(i,j,'*',q) <= 1 for i in Fruits for j in Colors for q in range(l))
color_row_con = m.addConstrs(z.sum(i,j,p,'*') <= 1 for i in Fruits for j in Colors for p in range(k))
for p in range(k):
    for q in range(l):
        error1Con = m.addConstr((d[p][q]-sum(r[i,j]*z[i,j,p,q] for i in Fruits for j in Colors) <= t))
        error2Con = m.addConstr((d[p][q]-sum(r[i,j]*z[i,j,p,q] for i in Fruits for j in Colors) >= -t))
m.update()

In [8]:
m.optimize()

Optimize a model with 2232 rows, 1393 columns and 8356 nonzeros
Variable types: 1 continuous, 1392 integer (1392 binary)
Coefficient statistics:
  Matrix range     [4e-02, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [6e-01, 1e+00]
Presolve removed 1394 rows and 0 columns
Presolve time: 0.03s
Presolved: 838 rows, 1393 columns, 5452 nonzeros
Variable types: 1 continuous, 1392 integer (1392 binary)

Root relaxation: infeasible, 164 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 infeasible    0               - infeasible      -     -    0s

Explored 0 nodes (164 simplex iterations) in 0.10 seconds
Thread count was 4 (of 4 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -
